In [186]:
#import libraries
import itertools
from fbprophet import Prophet
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [188]:
# select variables
pred_len = 10

In [189]:
#get the data
data = pd.read_csv("../data/Daily Bike Sharing.csv")
data.head(1)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,1/1/2011,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985


In [174]:
data.tail(1)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
730,731,12/31/2012,1,1,12,0,1,1,2,0.215833,0.223487,0.5775,0.154846,439,2290,2729


In [190]:
dataset = data.loc[:, ["dteday", "cnt", "holiday"]]

In [191]:
dataset.dteday = pd.to_datetime(dataset.dteday)

In [192]:
# renaming variable
dataset = dataset.rename(columns={'cnt': 'y'})
dataset = dataset.rename(columns={'dteday': 'ds'})

In [193]:
holiday_dates = dataset[dataset.holiday == 1].ds
holidays = pd.DataFrame({'holiday': 'holi',
                        'ds': pd.to_datetime(holiday_dates),
                        'lower_window': -3,
                        'upper_window': 1})

In [194]:
# removing holiday column
dataset = dataset.drop(columns="holiday")

In [157]:
#Training and test set
test_days = 31
training_set = dataset.iloc[:-test_days, :]
test_set = dataset.iloc[-test_days:, :]
test_set.head(1)

,ds,y,workingday,weathersit,temp,atemp,hum,windspeed
700,2012-12-01,5191,0,2,0.298333,0.316904,0.806667,0.059704


In [158]:
test_set.tail(1)

,ds,y,workingday,weathersit,temp,atemp,hum,windspeed
730,2012-12-31,2729,1,2,0.215833,0.223487,0.5775,0.154846


In [195]:
# Facebook Prophet model
m = Prophet(growth="linear",
                yearly_seasonality=True,
                weekly_seasonality=True,
                daily_seasonality=False,
                holidays=holidays,
                seasonality_mode="multiplicative",
                seasonality_prior_scale=10,
                holidays_prior_scale=20,
                changepoint_prior_scale=0.01)
m.fit(dataset)


In [196]:
#Create Future Dataframe
future = m.make_future_dataframe(periods = pred_len, include_history=False,
                                 freq = "D")
future.head(1)

,ds
0,2013-01-01


In [161]:
#merge regressors
future = pd.concat([future, dataset.iloc[:7,2:]],
                   axis = 1)
future.head(8)

,ds,workingday,weathersit,temp,atemp,hum,windspeed
0,2013-01-01,0,2,0.344167,0.363625,0.805833,0.160446
1,2013-01-02,0,2,0.363478,0.353739,0.696087,0.248539
2,2013-01-03,1,1,0.196364,0.189405,0.437273,0.248309
3,2013-01-04,1,1,0.200000,0.212122,0.590435,0.160296
4,2013-01-05,1,1,0.226957,0.229270,0.436957,0.186900
5,2013-01-06,1,1,0.204348,0.233209,0.518261,0.089565
6,2013-01-07,1,2,0.196522,0.208839,0.498696,0.168726


In [202]:
#forecast
forecast = m.predict(future)

In [200]:
forcast=forecast[['ds','yhat']].head(20)

In [204]:
forcast.head(20)

,ds,yhat
0,2013-01-01,2903.570789
1,2013-01-02,3029.694299
2,2013-01-03,3229.369238
3,2013-01-04,3285.213387
4,2013-01-05,3191.061456
5,2013-01-06,2740.334107
6,2013-01-07,3067.503494
7,2013-01-08,3447.122094
8,2013-01-09,3650.221322
9,2013-01-10,3916.376263


In [205]:
dataset.tail(9)

,ds,y
722,2012-12-23,1787
723,2012-12-24,920
724,2012-12-25,1013
725,2012-12-26,441
726,2012-12-27,2114
727,2012-12-28,3095
728,2012-12-29,1341
729,2012-12-30,1796
730,2012-12-31,2729


In [184]:
forcast.to_csv('../data/forcast_future/fbprophet.csv')